In [1]:
import os
os.environ["PROJECT_DIR"] = "test"
from generate_dataset_pipeline import generate_dataset
import numpy as np
from train import *
from sklearn.ensemble import RandomForestClassifier
from utils.plot_utils import plot_decision_boudaries
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from TabSurvey.models.saint import SAINT

In [6]:
config = {"model_name": "ft_transformer_regressor",
                  "model_type": "skorch",
                  "model__use_checkpoints": False,
                  "model__optimizer": "adamw",
                  "model__lr_scheduler": True,
                  "model__batch_size": 512,
                  "model__max_epochs": 300,
                  "model__module__activation": "reglu",
                  "model__module__normalization": "batchnorm",
                  "model__module__n_layers": 8,
                  "model__module__d": 256,
                  "model__module__d_hidden_factor": 2,
                  "model__module__hidden_dropout":  0.2,
                  "model__module__residual_dropout":  0.2,
                  "model__lr": 1e-3,
                  "model__optimizer__weight_decay":  1e-7,
                  "model__module__d_embedding": 128,
                  #"model__verbose": 100,
                  "model__max_epochs": 2,
                  "regression": True,
                  "data__regression": False,
                  "data__method_name": "real_data",
                  "data__categorical": False,
                  "data__keyword": "wine",
                  "transform__0__method_name": "gaussienize",
                  "transform__0__type": "quantile",
                  "transform__0__apply_on": "numerical",
                  #"transformed_target": True,
                  "max_train_samples": 10000}

CONFIG_DEFAULT = {"train_prop": 0.70,
                  "val_test_prop": 0.3,
                  "max_val_samples": 50000,
                  "max_test_samples": 50000}

config.update(CONFIG_DEFAULT)

In [15]:
rng = np.random.RandomState(32)
x_train, x_val, x_test, y_train, y_val, y_test, categorical_indicator = generate_dataset(config, rng)
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
args = AttrDict()
params = AttrDict()
args.update({"batch_size": 128,
             "val_batch_size": 128,
             "epochs": 2,
             "early_stopping_rounds": 10,
            "num_features": x_train.shape[1],
            "use_gpu": False,
             "data_parallel": False,
             "cat_idx": [],
             "num_classes": 1,
             "objective": "binary",#"regression",
             "model_name": "saint",
             "dataset": "superconduct"})
params.update({"depth": 6,
                      "heads": 8,
                      "dropout": 0.1,
               "dim": 5})
saint = SAINT(args=args,
              params=params)
saint.fit(x_train, y_train, x_val, y_val)

real_data
transforming data...
transform 0
gaussienize
Gaussienizing
{'batch_size': 128, 'val_batch_size': 128, 'epochs': 2, 'early_stopping_rounds': 10, 'num_features': 11, 'use_gpu': False, 'data_parallel': True, 'cat_idx': [], 'num_classes': 1, 'objective': 'binary', 'model_name': 'saint', 'dataset': 'superconduct'}
Using dim 5 and batch size 128


AttributeError: 'AttrDict' object has no attribute 'gpu_ids'

In [11]:
y_hat = saint.predict(x_test)

In [12]:
from sklearn.metrics import mean_squared_error, r2_score
print(r2_score(y_test, y_hat))
print(mean_squared_error(y_test, y_hat, squared=False))

-0.15488677988677968
0.5372528230760316


In [14]:
print(np.mean(y_test == y_hat))

0.7113594040968343
